In [186]:
%pip install yfiles_jupyter_graphs


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [187]:
from yfiles_jupyter_graphs import GraphWidget
import xml.etree.ElementTree as ET
import os 

In [188]:
namespaces = {
  'ODM': "http://www.cdisc.org/ns/odm/v1.3",
  'SDM':  "http://www.cdisc.org/ns/studydesign/v1.0"
}

def fully_qualified(ns, element):
  return "{%s}%s" % (ns, element)

def remove_ns(name):
  for k, v in namespaces.items():
    if v in name:
      return name.replace("{%s}" % (v), "")
  return name

node_types = { 
  fully_qualified(namespaces['ODM'], "ODM"): "root"
}
ignore_nodes = [ 
  fully_qualified(namespaces['SDM'], "Summary"), 
  fully_qualified(namespaces['ODM'], "Description"), 
  fully_qualified(namespaces['ODM'], "ItemGroupDef"),
  fully_qualified(namespaces['ODM'], "ItemDef") 
]
edge_nodes = [ 
  fully_qualified(namespaces['SDM'], "ActivityRef"), 
  fully_qualified(namespaces['ODM'], "FormRef"),
  fully_qualified(namespaces['SDM'], "ArmRef") 
]
preserve_oid_nodes = [ 
  fully_qualified(namespaces['SDM'], "ActivityDef"), 
  fully_qualified(namespaces['ODM'], "FormDef"),
  fully_qualified(namespaces['SDM'], "Arm") 
]

In [189]:
nodes = []
node_index = 0
edges = []
edge_index = 0
node_oid_map = {}
save_edges = []

def add_node(label, properties, node_type=None):
  global node_index
  global nodes
  node_index += 1
  properties['label'] = label
  if not node_type == None:
    properties['node_type'] = node_type
  nodes.append({ 'id': node_index, 'properties': properties })
  return node_index

def add_edge(label, start, end):
  global edge_index
  global edges
  edge_index += 1
  edges.append( {id: edge_index, 'start': start, 'end': end, 'properties': { 'label': label } })
  return edge_index

def save_edge(index, oid, label):
  global save_edges
  if not index == None:
    save_edges.append({ 'start': index, 'end_oid': oid, 'label': label })

def save_oid_def(oid, node_index):
  global node_oid_map
  node_oid_map[oid] = node_index

def find_oid_ref(properties):
  for k,v in properties.items():
    if "OID" in k:
      return v
  return None

In [190]:

def process_element(element, parent_index=None):
  element_index = None
  node_type = None
  if element.tag in edge_nodes:
    attribs = element.attrib
    label = remove_ns(element.tag)
    oid = find_oid_ref(attribs)
    save_edge(parent_index, oid, label)
  else:
    attribs = element.attrib
    label = remove_ns(element.tag)
    if element.tag in node_types:
      node_type = node_types[element.tag]
    element_index = add_node(label, attribs, node_type)
  if element.tag in preserve_oid_nodes:
    save_oid_def(attribs['OID'], element_index)
  for child in element:
    if not child.tag in ignore_nodes:
      child_index = process_element(child, element_index)
      if not child_index == None:
        child_edge_index = add_edge("child", element_index, child_index)
  return element_index

In [191]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/lzzt_trial.xml")
tree = ET.parse(file_path)

root = tree.getroot()

for item in root.findall('.'):
  process_element(item)

for edge in save_edges:
  oid = edge['end_oid']
  if oid in node_oid_map:
    add_edge(edge['label'], edge['start'], node_oid_map[oid])
    
#print("NODES:", nodes)
print("OID_MAP:", node_oid_map)
print("SAVE_EDGES:", save_edges)


OID_MAP: {'ARM.PLACEBO': 26, 'ARM.LOWDOSE': 27, 'ARM.HIGHDOSE': 28, 'ACT.TRIALSTART': 41, 'ACT.INFORMEDCONSENT': 42, 'ACT.PATIENTNUMBERASSIGNMENT': 43, 'ACT.DEMOGR_EDU_HABITS': 44, 'ACT.MMS': 45, 'ACT.HACHINSKI': 46, 'ACT.MEDICALHISTORY_01': 47, 'ACT.VITALSIGNS_01': 48, 'ACT.AMB_ECG_PLACEMENT_01': 49, 'ACT.AMB_ECG_REMOVAL_01': 50, 'ACT.PATIENTRANDOMIZATION': 51, 'ACT.VITALSIGNS_02': 52, 'ACT.TRIALFINISH': 53, 'ACT.ASSESS_FEVER': 54, 'ACT.ASSESS_HIGH_FEVER': 55, 'ACT.ECG_01': 56, 'ACT.CHEST_X-RAY': 57, 'ACT.MRI': 58, 'ACT.CT_SCAN': 59, 'ACT.TRT_VISIT_01_START': 60, 'ACT.TRT_VISIT_02_START': 61, 'ACT.TRT_VISIT_03_START': 62, 'ACT.TRT_VISIT_04_START': 63, 'ACT.TRT_VISIT_05_START': 64, 'ACT.TRT_VISIT_06_START': 65, 'ACT.TRT_VISIT_07_START': 66, 'ACT.TRT_VISIT_08_START': 67, 'ACT.TRT_VISIT_09_START': 68, 'ACT.TRT_VISIT_10_START': 69, 'ACT.ASSESS_HEADACHE': 70, 'FO.ECG': 147, 'FO.VS': 149, 'FO.INFORMEDCONSENT': 150, 'FO.INCLUSIONEXCLUSION': 151, 'FO.MMS': 152, 'FO.DEMOGR_EDUC_HABITS': 153, '

In [192]:
def custom_node_color(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'root':
      return 'black'
    elif node['properties']['node_type'] == 'anchor':
      return '#999999'
    elif node['properties']['node_type'] == 'condition':
      return '#999999'
    elif node['properties']['node_type'] == 'timepoint':
      return '#555555'
    elif node['properties']['node_type'] == 'visit':
      return '#c1141a'
    elif node['properties']['node_type'] == 'activity':
      return '#1555bd'
    elif node['properties']['node_type'] == 'bc':
      return '#c0d6e4'
    else:
      return 'white'
  else: 
    return 'white'

def custom_node_style(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'root':
      return { 'shape': 'round-rectangle' }
    elif node['properties']['node_type'] == 'anchor':
      return { 'shape': 'hexagon2' }
    elif node['properties']['node_type'] == 'condition':
      return { 'shape': 'diamond' }
    elif node['properties']['node_type'] == 'timepoint':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'visit':
      return { 'shape': 'circle' }
    elif node['properties']['node_type'] == 'activity':
      return { 'shape': 'circle' }
    elif node['properties']['node_type'] == 'bc':
      return { 'shape': 'circle' }
    else:
      return { 'shape': 'circle' }
  else: 
    return { 'shape': 'circle' }


In [193]:
w = GraphWidget()
w.set_directed(True)
w.hierarchic_layout()

w.set_nodes(nodes)
w.set_edges(edges)

w.set_node_color_mapping(custom_node_color)
w.set_node_styles_mapping(custom_node_style)
w

/Users/daveih/Library/Python/3.10/lib/python/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
keys must be str, int, float, bool or None, not builtin_function_or_method
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


GraphWidget(layout=Layout(height='500px', width='100%'))